In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
train_df = pd.read_csv('/kaggle/input/jan2022playground/Playground_folds_train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2022/test.csv')
sub_df =  pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/sample_submission.csv")

---

# 1. Overview of Data

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

> * <b>Outliers exist in the train data set</b>

---
# 2. Exploratory Data Analysis

In [ ]:
train_df.head(3)

In [ ]:
prods = train_df.groupby(['product'])['num_sold'].sum().reset_index()
prods

In [ ]:

plt.figure()
sns.barplot(x = prods['product'], y = prods['num_sold'])

plt.title("Products Sold by category")
plt.xlabel('Kaggle Merchandies')
plt.xticks(size=15)
plt.yticks(size=15)
plt.ylabel('No. of sold')

plt.show()

In [ ]:
store_selling = train_df.groupby(['store'])['num_sold'].sum().reset_index()
store_selling

In [ ]:
plt.figure()
sns.barplot(x = store_selling['store'], y = store_selling['num_sold'])

plt.title("Merchandise Sold by Store")
plt.xlabel('Store')
plt.xticks(size=15)
plt.yticks(size=15)
plt.ylabel('No. of sold by Store')

plt.show()

In [ ]:
ctryWise = train_df.groupby(['country'])['num_sold'].sum().reset_index()
ctryWise

In [ ]:
plt.figure()
sns.barplot(x = ctryWise['country'], y = ctryWise['num_sold'])

plt.title("Merchandise Sold by Store")
plt.xlabel('Country')
plt.xticks(size=15)
plt.yticks(size=15)
plt.ylabel('No. of sold Item')

plt.show()

---

# Preprocessing and Model Training

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#preparing the dataset through folds

features = [col for col in train_df.columns if col not in ('num_sold', 'kfold')]
obj_col = [col for col in train_df.columns if 'cat' in col]

test_df = test_df[features]

In [ ]:
numerical_transformer = SimpleImputer(strategy = 'most_frequent') # Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer' , SimpleImputer(strategy = 'most_frequent')),
     ('oneHot' , OneHotEncoder(handle_unknown='ignore'))
]) # Your code here

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, obj_col)
    ])

# Define model
# model = RandomForestRegressor()
model1 = GradientBoostingRegressor(n_estimators=50,
            learning_rate=0.005, max_depth=6)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
# preds = []

# for i in range(5):
#     X_train = train_df[train_df['kfold'] != i].reset_index(drop=False)
#     X_valid = train_df[train_df['kfold'] == i].reset_index(drop=False)
#     X_test = test_df.copy()
    
#     y_train  = X_train['num_sold']
#     y_valid  = X_valid['num_sold']
    
#     X_train = X_train[features]
#     X_valid = X_valid[features]
    
#     print(X_train)
#     # Bundle preprocessing and modeling code in a pipeline
#     my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                   ('model', model1)
#                                  ])
    
#     # Preprocessing of training data, fit model 
# #     my_pipeline.fit(X_train, y_train)

#     # # Preprocessing of validation data, get predictions
#     predictions = my_pipeline.predict(X_test)
#     preds.append(predictions)
    

# smape = smape(y_valid, preds[:5260])
    
    

In [ ]:
X_test

In [ ]:
y_train.column = ["num_sold"]

In [ ]:
y_train

In [ ]:
# predictions
sub_df['num_sold'] = preds
sub_df.to_csv('submission.csv', index=False)

----

fold work end here

In [ ]:
X = train_df.Error
y = train_df.num_sold
X.drop(['num_sold'], axis=1, inplace=True)
y.columns = ['num_sold']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


In [ ]:
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

In [ ]:
numerical_transformer = SimpleImputer(strategy = 'most_frequent') # Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer' , SimpleImputer(strategy = 'most_frequent')),
     ('oneHot' , OneHotEncoder(handle_unknown='ignore'))
]) # Your code here

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
# model = RandomForestRegressor()
model1 = GradientBoostingRegressor(n_estimators=50,
            learning_rate=0.005, max_depth=6)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


In [ ]:
# # Bundle preprocessing and modeling code in a pipeline
# my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', model1)
#                              ])

# # Preprocessing of training data, fit model 
# my_pipeline.fit(X_train, y_train)

# # # Preprocessing of validation data, get predictions
# preds = my_pipeline.predict(test_df)

# smape = smape(y_valid, preds[:5260])

In [ ]:
smape

In [ ]:
predictions = []
predictions.append(preds)

In [ ]:
# predictions
sub_df['num_sold'] = preds
sub_df.to_csv('submission.csv', index=False)

In [ ]:
# import optuna

In [ ]:

# preds = []

# def run(trial):
# #     i = 0
#     learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
# #     reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
#     reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
#     subsample = trial.suggest_float("subsample", 0.1, 1.0)
#     colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
#     max_depth = trial.suggest_int("max_depth", 1, 12)
    
    
#     model = GradientBoostingRegressor(
#             n_estimators=1000,
#             learning_rate=learning_rate,
#             subsample=subsample,
            
#             max_depth=max_depth)
    
#     my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', model)
#                              ])

#     # Preprocessing of training data, fit model 
#     my_pipeline.fit(X_train, y_train)

#     # # Preprocessing of validation data, get predictions
#     preds = my_pipeline.predict(test_df)

#     smape(y_valid, preds[:5260])
# #     print(smape)

In [ ]:
# study = optuna.create_study(direction="minimize")
# study.optimize(run, n_trials=5)

In [ ]:
# study.best_params